In [1]:
import pandas as pd
import numpy as np
df = pd.read_excel(r'C:\Users\USER\Desktop\web-scraping-\final.xlsx')

In [2]:
import gensim
from gensim.models.ldamulticore import LdaMulticore
from gensim import corpora, models
import pyLDAvis.gensim_models
from nltk.corpus import stopwords
import string
from nltk.stem.wordnet import WordNetLemmatizer

import warnings
warnings.simplefilter('ignore')
from itertools import chain

C:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
#import nltk
#nltk.download('all')

In [4]:
df.head()

,topic,title,article,date,link
0,BUSINESS,"MAHB faces double whammy of interstate ban, Tu...",KUALA LUMPUR: Affin Hwang Capital expects Mala...,"May 4, 2021 @ 10:40am",https://www.nst.com.my/business/2021/05/687692...
1,BUSINESS,RM opens at 4.09 against USD,KUALA LUMPUR: The ringgit opened easier agains...,"May 3, 2021 @ 10:09am",https://www.nst.com.my/business/2021/05/687398...
2,COLUMNISTS,Biden needs fresh thinking and more than one term,FORGET about the first 100 days. US President ...,"May 3, 2021 @ 12:10am",https://www.nst.com.my/opinion/columnists/2021...
3,SUNDAY VIBES,MONEY THOUGHTS: Ikigai — The uncommon key to c...,HAVE you noticed how life's getting tougher on...,"May 2, 2021 @ 8:30am",https://www.nst.com.my/lifestyle/sunday-vibes/...
4,BUSINESS,"Labuan FSA appoints new chairman, director gen...",KUALA LUMPUR: Labuan Financial Services Author...,"Apr 30, 2021 @ 4:15pm",https://www.nst.com.my/business/2021/04/686731...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107 entries, 0 to 106
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   topic    107 non-null    object
 1   title    107 non-null    object
 2   article  107 non-null    object
 3   date     107 non-null    object
 4   link     107 non-null    object
dtypes: object(5)
memory usage: 4.3+ KB


In [6]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

def clean(text):
    stop_free = ' '.join([word for word in text.lower().split() if word not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = ' '.join([lemma.lemmatize(word) for word in punc_free.split()])
    return normalized.split()

In [7]:
df['article_clean']=df['article'].apply(clean)

In [8]:
dictionary = corpora.Dictionary(df['article_clean'])
print(dictionary.num_nnz)

20580


In [9]:
doc_term_matrix = [dictionary.doc2bow(doc) for doc in df['article_clean'] ]
print(len(doc_term_matrix))

107


In [10]:
lda = gensim.models.ldamodel.LdaModel

In [11]:
num_topics= 7
%time ldamodel = lda(doc_term_matrix,num_topics=num_topics,id2word=dictionary,passes=200,minimum_probability=0)

Wall time: 33.6 s


In [12]:
ldamodel.print_topics(num_topics=num_topics)

[(0,
  '0.011*"government" + 0.009*"country" + 0.008*"economic" + 0.007*"covid19" + 0.007*"also" + 0.006*"people" + 0.006*"malaysia" + 0.005*"digital" + 0.004*"pandemic" + 0.004*"2020"'),
 (1,
  '0.012*"said" + 0.012*"per" + 0.012*"cent" + 0.008*"malaysia" + 0.006*"fund" + 0.006*"year" + 0.006*"2020" + 0.006*"investor" + 0.006*"market" + 0.006*"investment"'),
 (2,
  '0.014*"food" + 0.007*"said" + 0.005*"u" + 0.004*"country" + 0.004*"million" + 0.004*"consumer" + 0.004*"quarter" + 0.003*"bank" + 0.003*"policy" + 0.003*"need"'),
 (3,
  '0.012*"per" + 0.012*"cent" + 0.009*"said" + 0.006*"government" + 0.006*"also" + 0.005*"billion" + 0.005*"economy" + 0.005*"year" + 0.005*"bond" + 0.005*"currency"'),
 (4,
  '0.018*"said" + 0.008*"malaysia" + 0.005*"also" + 0.005*"new" + 0.005*"per" + 0.005*"digital" + 0.005*"cent" + 0.005*"global" + 0.004*"economy" + 0.004*"economic"'),
 (5,
  '0.025*"ringgit" + 0.024*"dollar" + 0.024*"u" + 0.018*"currency" + 0.017*"local" + 0.014*"market" + 0.014*"said" 

In [13]:
lda_display = pyLDAvis.gensim_models.prepare(ldamodel, doc_term_matrix, dictionary, sort_topics=False, mds='mmds')
pyLDAvis.display(lda_display)

In [14]:
lda_corpus = ldamodel[doc_term_matrix]

In [15]:
[doc for doc in lda_corpus]

[[(0, 0.0006377429),
  (1, 0.00063829334),
  (2, 0.8339847),
  (3, 0.0006382162),
  (4, 0.16282506),
  (5, 0.0006386765),
  (6, 0.00063731894)],
 [(0, 0.001014644),
  (1, 0.0010153223),
  (2, 0.0010151279),
  (3, 0.0010163785),
  (4, 0.3031447),
  (5, 0.6917788),
  (6, 0.0010150454)],
 [(0, 0.0001546966),
  (1, 0.00015469162),
  (2, 0.4269697),
  (3, 0.00015475719),
  (4, 0.00015456355),
  (5, 0.00015473636),
  (6, 0.57225686)],
 [(0, 0.0002580439),
  (1, 0.00025815362),
  (2, 0.99845165),
  (3, 0.00025803578),
  (4, 0.00025794443),
  (5, 0.00025799347),
  (6, 0.00025822473)],
 [(0, 0.001017467),
  (1, 0.9939036),
  (2, 0.0010158649),
  (3, 0.0010161618),
  (4, 0.001015982),
  (5, 0.0010152984),
  (6, 0.0010156644)],
 [(0, 0.0006820119),
  (1, 0.000682303),
  (2, 0.00068193144),
  (3, 0.000682745),
  (4, 0.40144828),
  (5, 0.595141),
  (6, 0.00068170566)],
 [(0, 0.000809538),
  (1, 0.00080891186),
  (2, 0.00080874376),
  (3, 0.00080891943),
  (4, 0.0008111187),
  (5, 0.0008110404),
  (

In [16]:
scores = list(chain(*[[score for topic_id,score in topic] \
                      for topic in [doc for doc in lda_corpus]]))

threshold = sum(scores)/len(scores)
print(threshold)

0.14285714156346496


In [17]:
cluster1 = [j for i,j in zip(lda_corpus,df.index) if i[0][1] > threshold]
cluster2 = [j for i,j in zip(lda_corpus,df.index) if i[1][1] > threshold]
cluster3 = [j for i,j in zip(lda_corpus,df.index) if i[2][1] > threshold]
cluster4 = [j for i,j in zip(lda_corpus,df.index) if i[3][1] > threshold]
cluster5 = [j for i,j in zip(lda_corpus,df.index) if i[4][1] > threshold]

In [18]:
df.iloc[cluster1]

,topic,title,article,date,link,article_clean
13,WORLD,Turkish crypto founder flees with reported $2bil,ISTANBUL: Turkish prosecutors on Thursday open...,"Apr 22, 2021 @ 9:10pm",https://www.nst.com.my/world/world/2021/04/684...,"[istanbul, turkish, prosecutor, thursday, open..."
29,NATION,Yemeni civil war more devastating than Covid-1...,KUALA LUMPUR: The raging civil war in Yemen is...,"Mar 25, 2021 @ 5:24pm",https://www.nst.com.my/news/nation/2021/03/676...,"[kuala, lumpur, raging, civil, war, yemen, dev..."
45,NATION,PM's 'Setahun Malaysia Prihatin' keynote address,KUALA LUMPUR: Prime Minister Tan Sri Muhyiddin...,"Mar 1, 2021 @ 5:54pm",https://www.nst.com.my/news/nation/2021/03/670...,"[kuala, lumpur, prime, minister, tan, sri, muh..."
53,BOTS,"#TECH: Nyan Cat GIF fetched nearly US$500,000 ...",WHO would have thought that an animated flying...,"Feb 24, 2021 @ 8:49pm",https://www.nst.com.my/lifestyle/bots/2021/02/...,"[would, thought, animated, flying, cat, leavin..."
62,CRIME & COURTS,Crypto-currency investment scheme lands 2 busi...,IPOH: Two businessmen were charged at the Magi...,"Feb 2, 2021 @ 11:35am",https://www.nst.com.my/news/crime-courts/2021/...,"[ipoh, two, businessmen, charged, magistrate, ..."
66,GOVERNMENT / PUBLIC POLICY,Tengku Zafrul: No total lockdown even in worst...,KUALA LUMPUR: Finance Minister Tengku Datuk Se...,"Jan 29, 2021 @ 2:51pm",https://www.nst.com.my/news/government-public-...,"[kuala, lumpur, finance, minister, tengku, dat..."
97,WORLD,"Singapore, Osaka no longer world's most expens...",LONDON: Paris and Zurich have joined Hong Kong...,"Nov 20, 2020 @ 11:17am",https://www.nst.com.my/world/world/2020/11/642...,"[london, paris, zurich, joined, hong, kong, ye..."


In [19]:
df.iloc[cluster5]

,topic,title,article,date,link,article_clean
0,BUSINESS,"MAHB faces double whammy of interstate ban, Tu...",KUALA LUMPUR: Affin Hwang Capital expects Mala...,"May 4, 2021 @ 10:40am",https://www.nst.com.my/business/2021/05/687692...,"[kuala, lumpur, affin, hwang, capital, expects..."
1,BUSINESS,RM opens at 4.09 against USD,KUALA LUMPUR: The ringgit opened easier agains...,"May 3, 2021 @ 10:09am",https://www.nst.com.my/business/2021/05/687398...,"[kuala, lumpur, ringgit, opened, easier, u, do..."
5,BUSINESS,RM opens at 4.09 against USD,KUALA LUMPUR: The ringgit opened firmer agains...,"Apr 30, 2021 @ 10:43am",https://www.nst.com.my/business/2021/04/686620...,"[kuala, lumpur, ringgit, opened, firmer, u, do..."
8,BUSINESS,RM opens marginally lower against USD,KUALA LUMPUR: The ringgit opened marginally lo...,"Apr 27, 2021 @ 10:38am",https://www.nst.com.my/business/2021/04/685755...,"[kuala, lumpur, ringgit, opened, marginally, l..."
9,BUSINESS,Revising SPV2030 targets a sensible move: MARC,KUALA LUMPUR: It is sensible for the governmen...,"Apr 26, 2021 @ 2:44pm",https://www.nst.com.my/business/2021/04/685508...,"[kuala, lumpur, sensible, government, revise, ..."
19,BUSINESS,Luno Malaysia manages over RM1bil DAUM,"KUALA LUMPUR: Global cryptocurrency company, L...","Apr 14, 2021 @ 11:51am",https://www.nst.com.my/business/2021/04/682203...,"[kuala, lumpur, global, cryptocurrency, compan..."
22,BUSINESS,"Export conversion rule, other liberalisation m...",KUALA LUMPUR: Bank Negara Malaysia has announc...,"Mar 31, 2021 @ 10:15am",https://www.nst.com.my/business/2021/03/678354...,"[kuala, lumpur, bank, negara, malaysia, announ..."
23,CRIME & COURTS,"'Nicky Gang' busted, Datuk Seri Nicky still mi...",KUALA LUMPUR: The police have arrested 68 indi...,"Mar 30, 2021 @ 7:21pm",https://www.nst.com.my/news/crime-courts/2021/...,"[kuala, lumpur, police, arrested, 68, individu..."
27,CRIME & COURTS,Hunt on for Bitcoin mining culprit behind TNB'...,MELAKA: Police here are hunting for a man beli...,"Mar 26, 2021 @ 7:39pm",https://www.nst.com.my/news/crime-courts/2021/...,"[melaka, police, hunting, man, believed, main,..."
30,BUSINESS,Ringgit slips to 4.1330 versus USD at opening ...,KUALA LUMPUR: The ringgit slipped further vers...,"Mar 25, 2021 @ 10:05am",https://www.nst.com.my/business/2021/03/676818...,"[kuala, lumpur, ringgit, slipped, versus, u, d..."
